In [1]:
%reset -f 
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import UnivariateSpline
import time
import derivatives as d
from chameleon import Chameleon

In [2]:
chameleon = Chameleon(E=10)

In [3]:
active_stress = np.linspace(1, 10, chameleon.n_elems)

In [4]:
# active_stress = (1/10)*active_stress**2

In [5]:
active_stress

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

## Is this is too slow? In order to do RL I want to be able to simulate much faster than this. (Jax?)

### could also simulate for a shorter time when I want to.

I could almost already build this as an RL environment. Just need to wrap this around a gym environment, reset the thing according to the most recent position and have each specification of torque account for a smaller period of simulation time. Maybe do 1000 steps of sim instead of 10000.



In [6]:
# %%time
# chameleon.step(active_stress)

In [7]:
%%time
chameleon.forward_simulate(active_stress, sim_steps=1000)

ValueError: x must be increasing if s > 0

# Error indicates that the element on the end is not farther out than the element that is next to last. 

I had this issue before and I am not sure how to solve it right now.

In [8]:
chameleon.pos_f

array([0.        , 0.12011111, 0.23122222, 0.34233333, 0.45344444,
       0.56455556, 0.67566667, 0.78677778, 0.89788889, 0.83      ])

In [9]:
chameleon.disp_current

array([ 0.   ,  0.009,  0.009,  0.009,  0.009,  0.009,  0.009,  0.009,
        0.009, -0.17 ])

In [11]:
dx = chameleon.pos_0[-1] - chameleon.pos_0[-2]

In [13]:
factor = 2*dx*active_stress[-1]/chameleon.E

In [14]:
factor

0.22222222222222232

In [15]:
chameleon.disp_current[-1]

-0.16999999999999993

In [16]:
chameleon.disp_previous[-1]

0.0

# some hacky code to store the position history for later visualization.


In [ ]:
history = []
for i in range(len(chameleon.position_history)):
    x_p = chameleon.position_history[i]
    y = np.zeros(10)
    ar = np.vstack((x_p, y)).T
    history.append(ar)

with open("history.txt", "wb") as fp:
    pickle.dump(history, fp)

